# Test the sqldb

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings('ignore')

In [2]:
db_path=str(here('data'))+'/sqldb.db'
db=SQLDatabase.from_uri(f'sqlite:///{db_path}')

In [3]:
print(db.dialect)
print(db.get_usable_table_names())
db.run('select * from artist limit 10;')

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# test the access to the environment variables 

In [23]:
from dotenv import load_dotenv
import os
print('Environment variables are loaded',load_dotenv())
#print('test by reading a variable:',os.getenv())

Environment variables are loaded True


# test GPT model

In [17]:
from openai import AzureOpenAI

message=[
    {'role':'system','content':str('you are helpful assistant')
     
     },
     {'role':'user','content':str('say hello in hindi')}
]

client=AzureOpenAI(
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OpenAI_ENDPOINT')
)

response=client.chat.completions.create(
    model=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    messages=message
)
print(response.choices[0].message.content)

Namaste


# SQL query chain

In [25]:
from langchain_community.chat_models import AzureChatOpenAI

model_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL')
azure_openai_api_key=os.environ['AZURE_OPENAI_API_KEY']
azure_openai_endpoint=os.environ['AZURE_OpenAI_ENDPOINT']

llm=AzureChatOpenAI(
    openai_api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.2
)

In [46]:
from langchain.chains import create_sql_query_chain

chain=create_sql_query_chain(llm,db)
response=chain.invoke({'question':'print the name and month of employees whose birdate is on march'})
print(response)

SELECT "FirstName", "LastName", strftime('%m', "BirthDate") AS "BirthMonth" 
FROM "Employee" 
WHERE strftime('%m', "BirthDate") = '03'


# execute the query to on the database

In [45]:
db._execute(response)

[{'FirstName': 'Steve', 'BirthMonth': None}]

In [47]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

# add query SQLDatabase tool to the chain

In [50]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query=create_sql_query_chain(llm,db)
execute_query=QuerySQLDataBaseTool(db=db)

chain=write_query|execute_query
chain.invoke({'question':'list the employee names whose birthdays are in march'})

"[('Steve', 'Johnson')]"

# Answer the question in a user friendly manner

In [59]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt=PromptTemplate.from_template(
    """
    Given the following user question, coresponding SQL query and SQL resutls, answer the user question
Question:{question}
SQL Query: {query}
SQL Results: {result}
Answer: 

"""
)

answer = answer_prompt|llm|StrOutputParser()
chain=(
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter('query')|execute_query
    )|answer

)

chain.invoke({'question':'give all the details of employees can you give the result in a table'})

'Here are the details of the employees in a table:\n\n| EmployeeId | LastName | FirstName | Title                | ReportsTo | BirthDate           | HireDate            | Address                  | City       | State | Country | PostalCode | Phone              | Fax                | Email                   |\n|------------|----------|-----------|----------------------|-----------|---------------------|---------------------|--------------------------|-------------|-------|---------|------------|--------------------|-------------------|-------------------------|\n| 1          | Adams    | Andrew    | General Manager      | None      | 1962-02-18 00:00:00 | 2002-08-14 00:00:00 | 11120 Jasper Ave NW      | Edmonton    | AB    | Canada  | T5K 2N1    | +1 (780) 428-9482 | +1 (780) 428-3457 | andrew@chinookcorp.com   |\n| 2          | Edwards  | Nancy     | Sales Manager        | 1         | 1958-12-08 00:00:00 | 2002-05-01 00:00:00 | 825 8 Ave SW             | Calgary     | AB    | Canada  |

In [57]:
chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delim

# implement Agents 

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL agent are:
- it can answer specific to the database schema as well as database content like describing a specific table 
- it can recover from errors by running a generated query, catching the traceback and regenrating it correctly
- it can answer questions that require multiple dependancies
- it will save tokens by only considering the schema from relevant tables

to initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:
- create and exexute queries 
- check query syntax
- retrieve table desc 


In [62]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor=create_sql_agent(llm,db=db,agent_type='openai-tools',verbose=True)

In [64]:
agent_executor.invoke(
    {
        'input':'list the total sales per country. which countrys customers spent the most?'
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice, Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José

{'input': 'list the total sales per country. which countrys customers spent the most?',
 'output': 'The country whose customers spent the most is the USA, with a total sales amount of $523.06.'}

In [66]:
agent_executor.invoke(
    {'input':'what is the average total in invoice'}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice, InvoiceLine'}`



CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2021-01-01 00:00:00	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	1.98
2	4	2021-01-02 00:00:00	Ullevålsveien 14	Oslo	None	Norway	0171	3.96
3	8	2021-01-03 00:

{'input': 'what is the average total in invoice',
 'output': 'The average total in invoices is approximately 5.65.'}

In [87]:
import pandas as pd
pd.DataFrame(db._execute('select * from invoice'))

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2021-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2021-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2021-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2021-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2021-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2025-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2025-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2025-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2025-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [80]:
invoiceDF=pd.DataFrame(db._execute('select * from invoice'))


In [82]:
sum(invoiceDF['Total'])/len(invoiceDF['Total'])

5.651941747572825